# Lab 6.2: Fancy Softmax Classification

Edited By Steve Ive.

Reference from Seungjae Lee

https://github.com/deeplearningzerotoall/PyTorch/blob/master/lab-06_2_fancy_softmax_classification.ipynb

## Imports

In [101]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [102]:
torch.manual_seed(1)

## Cross Entropy Loss with ```torch.nn.functional```
PyTorch has ```F.log_softmax()```

In [103]:
z = torch.rand(3, 5, requires_grad = True)
pred = F.softmax(z, dim =1)

y = torch.randint(5, (3,)).long()

y_one_hot = torch.zeros_like(pred)
y_one_hot = y_one_hot.scatter_(1, y.unsqueeze(1), 1)

In [104]:
#Low Level
torch.log(F.softmax(z, dim = 1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward>)

In [105]:
#High Level
F.log_softmax(z, dim = 1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward>)

PyTorch also has ```F.nll_loss()``` function that computes the negative loss likelihood.

In [106]:
F.cross_entropy(z, y)

tensor(1.4689, grad_fn=<NllLossBackward>)

## Data

In [107]:
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype =np.float32)

In [108]:
x_train = torch.FloatTensor(xy[:, 0:-1])
y_train = torch.LongTensor(xy[:, -1])

In [109]:
print(x_train.shape)
print(y_train.shape)
print(len(x_train))
print(x_train[:5])

torch.Size([101, 16])
torch.Size([101])
101
tensor([[1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 4., 0., 0., 1.],
        [1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 4., 1., 0., 1.],
        [0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0.],
        [1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 4., 0., 0., 1.],
        [1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 4., 1., 0., 1.]])


In [110]:
nb_classes = 7
y_one_hot = torch.zeros((len(x_train), nb_classes))
y_one_hot = y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
# or F.one_hot(y_train)

## Training with ```F.cross.entropy```

In [111]:
# Model initialize
W = torch.zeros((16, 7), requires_grad = True)
b = torch.zeros(1, requires_grad=True)

#Set Optimizer
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000

for epoch in range(nb_epochs):

    #prediction
    pred = x_train.matmul(W) + b

    #cost
    cost = F.cross_entropy(pred, y_train)

    #Reduce cost
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        accuracy = (F.one_hot(torch.argmax(F.softmax(pred, dim=1), dim=1)) == F.one_hot(y_train)).float().mean()*100
        print('Epoch {:2d}/{} accuracy: {} cost: {:.6f}'.format(epoch, nb_epochs, accuracy, cost))


Epoch  0/1000 accuracy: 14.285715103149414 cost: 1.945910
Epoch 100/1000 accuracy: 96.3224868774414 cost: 0.471836
Epoch 200/1000 accuracy: 98.01980590820312 cost: 0.326327
Epoch 300/1000 accuracy: 98.58557891845703 cost: 0.257839
Epoch 400/1000 accuracy: 98.58557891845703 cost: 0.215762
Epoch 500/1000 accuracy: 98.58557891845703 cost: 0.186603
Epoch 600/1000 accuracy: 99.1513442993164 cost: 0.164898
Epoch 700/1000 accuracy: 99.4342269897461 cost: 0.147955
Epoch 800/1000 accuracy: 100.0 cost: 0.134279
Epoch 900/1000 accuracy: 100.0 cost: 0.122962


## High-level Implementaion with ```nn.Module```

In [112]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(16, 7)

    def forward(self, x):
        return self.linear(x)

In [113]:
model = SoftmaxClassifierModel()

In [114]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000

for epoch in range(nb_epochs):

    #prediction
    pred = model(x_train)

    #cost
    cost = F.cross_entropy(pred, y_train)

    #Reduce cost
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        accuracy = (F.one_hot(torch.argmax(F.softmax(pred, dim=1), dim =1)) == F.one_hot(y_train)).float().mean()*100
        print('Epoch {:2d}/{} Accuracy: {:.3f} Cost: {:.6f}'.format(epoch, nb_epochs, accuracy, cost))

Epoch  0/1000 Accuracy: 74.257 Cost: 1.919160
Epoch 100/1000 Accuracy: 96.605 Cost: 0.468405
Epoch 200/1000 Accuracy: 97.737 Cost: 0.320585
Epoch 300/1000 Accuracy: 98.586 Cost: 0.248953
Epoch 400/1000 Accuracy: 98.586 Cost: 0.204819
Epoch 500/1000 Accuracy: 99.151 Cost: 0.174506
Epoch 600/1000 Accuracy: 99.151 Cost: 0.152248
Epoch 700/1000 Accuracy: 100.000 Cost: 0.135139
Epoch 800/1000 Accuracy: 100.000 Cost: 0.121543
Epoch 900/1000 Accuracy: 100.000 Cost: 0.110461
